In [2]:
import numpy as np
import pandas as pd
import h5py
import os
from tqdm.notebook import tqdm

from methods.io_things import tar_to_downscaled_models
from methods.plotting import plotly_plot

# CURR_DIR = os.getcwd()

ROCKS = pd.read_excel('rock_properties.xlsx', index_col=0)
ROCKS

,Lithology,Lithology class,Genetic class,Mean density (g cm−3),Std density,Mean permeability (Log10(mD)),Std permeability
Fels_Dyke_Sill,Felsic_Dyke_Sill,Dyke,Intrusive,2.612593,0.090526,1.56,0.58
Maf_Dyke_Sill,Mafic_Dyke_Sill,Dyke,Intrusive,2.793914,0.015760,1.56,0.58
Granite,Granite,Plug,Intrusive,2.691577,0.094590,1.56,0.58
Peridotite,Peridotite,Plug,Intrusive,2.851076,0.154478,1.56,0.58
Porphyry,Porphyry,Plug,Intrusive,2.840024,0.128972,1.56,0.58
Pyxenite_Hbnd,Pyxenite_Hbndite,Plug,Intrusive,3.194379,0.253323,1.56,0.58
Gabbro,Gabbro,Plug,Intrusive,3.004335,0.159719,1.56,0.58
Diorite,Diorite,Plug,Intrusive,2.851608,0.134657,1.56,0.58
Syenite,Syenite,Plug,Intrusive,2.685824,0.115078,1.56,0.58
Amphib,Amphibolite,Met_strat,Metamorphic,2.875933,0.142164,2.00,0.57


In [ ]:
PATH_TO_MODELS = 'D:/geological_models/' # path to folder with TAR files like "DYKE_FOLD_FOLT" and others

NMODELS = 300 # of each type. Total number of models woul be = 343 * NMODELS. Put it over 9000 to process all models

yield_path = tar_to_downscaled_models(PATH_TO_MODELS, NMODELS, fname='downscaled_models')

In [9]:
yield_path = 'downscaled_models_02_26_2024__16_06_58.h5'

with h5py.File(yield_path, 'r') as f:
    perm = f['perm'][2]
    # dens = f['dens'][2]

plotly_plot(perm)


todo
распихать по функциям
лист в расходе
неоднородное уравнение
параметры в трензорах
функиця подгонки давления (приёмистость)